In [22]:
import plotly.graph_objects as go
from scipy.spatial import Delaunay
import numpy as np

In [23]:
fig = go.Figure(data=[go.Scatter3d(
    x=[1, 2, 3], 
    y=[4, 5, 6], 
    z=[7, 8, 9], 
    mode='lines+markers',
    marker=dict(size=5, color='blue'),
    line = dict(color="red", width=2)
)])

fig.update_layout(
    width = 800,
    height = 800
)

fig.show()

In [24]:
def plot_rectangle():

    # Vertices of the rectangular prism
    vertices = [
        [3, 3, 0], [5, 3, 0], [5, 5, 0], [3, 5, 0],  # Bottom face
        [3, 3, 10], [5, 3, 10], [5, 5, 10], [3, 5, 10]  # Top face
    ]

    # Lists of x, y, z coordinates
    x = [vertex[0] for vertex in vertices]
    y = [vertex[1] for vertex in vertices]
    z = [vertex[2] for vertex in vertices]

    # Define the 12 triangles composing the rectangular prism

    i = [0, 1, 7, 7, 5, 5, 7, 7, 0, 2, 4, 6]
    j = [1, 5, 3, 6, 1, 6, 3, 4, 1, 3, 5, 7]
    k = [4, 4, 2, 2, 2, 2, 0, 0, 2, 0, 6, 4]

    # Create a 3D mesh plot
    fig = go.Figure(data=[go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=i,
        j=j,
        k=k,
        opacity=0.5,
        color='red'
    )])

    # Update layout for better visualization
    fig.update_layout(scene=dict(
        xaxis=dict(nticks=4, range=[2, 6]),
        yaxis=dict(nticks=4, range=[2, 6]),
        zaxis=dict(nticks=6, range=[0, 11])
    ))

    # Show the figure
    fig.show()

In [25]:
plot_rectangle()

def initial_plot(delivary_locations, obstacles, dp=None, route=None):
    fig = go.Figure()

    # Plot each building
    for building in obstacles.values():
        x = [point[0] for point in building]
        y = [point[1] for point in building]
        z = [point[2] for point in building]

        # Bottom face
        fig.add_trace(go.Mesh3d(
            x=x,
            y=y,
            z=[0] * len(x),
            color='red',
            opacity=0.80
        ))

        # Top face
        fig.add_trace(go.Mesh3d(
            x=x,
            y=y,
            z=z,
            color='red',
            opacity=0.80
        ))

        # Side faces
        for i in range(len(x)):
            j = (i + 1) % len(x)

            # Define two triangles for each side face
            x_side = [x[i], x[j], x[j], x[i]]
            y_side = [y[i], y[j], y[j], y[i]]
            z_side = [0, 0, z[j], z[i]]

            i_next = (i + 1) % len(x)
            triangles = [
                (i, i_next, i_next + len(x)),  # Triangle 1
                (i, i_next + len(x), i + len(x))  # Triangle 2
            ]

            fig.add_trace(go.Mesh3d(
                x=x + x,
                y=y + y,
                z=[0] * len(x) + z,
                i=[tri[0] for tri in triangles],
                j=[tri[1] for tri in triangles],
                k=[tri[2] for tri in triangles],
                color='red',
                opacity=0.50
            ))

    # Delivery points
    loc_x = [i for i, j, k in delivary_locations]
    loc_y = [j for i, j, k in delivary_locations]
    loc_z = [k for i, j, k in delivary_locations]

    fig.add_trace(go.Scatter3d(
        x=loc_x,
        y=loc_y,
        z=loc_z,
        mode='markers',
        marker=dict(size=5, color='blue')
    ))

    # Plot dp
    if dp:
        fig.add_trace(go.Scatter3d(
            x=[dp[0]],
            y=[dp[1]],
            z=[dp[2]],
            mode='markers',
            marker=dict(size=8, color='orange')
        ))

    # Plotting route
    if route:
        x_axis = [i for i, j, k in route]
        y_axis = [j for i, j, k in route]
        z_axis = [k for i, j, k in route]

        fig.add_trace(go.Scatter3d(
            x=x_axis,
            y=y_axis,
            z=z_axis,
            mode='lines',
            line=dict(color='gray', width=2)
        ))

    # Update layout for better view
    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Height',
        aspectmode='cube'
    ))

    # Show plot
    fig.show()

In [26]:
def triangulate_polygon(x, y):
    points2D = np.vstack([x, y]).T
    tri = Delaunay(points2D)
    return tri.simplices

def initial_plot(delivery_locations, obstacles, dp=None, route=None):
    fig = go.Figure()

    # Plot each building
    for building in obstacles.values():
        x = [point[0] for point in building]
        y = [point[1] for point in building]
        z = [point[2] for point in building]

        # Triangulate top and bottom faces
        if len(x) > 2:  # Need at least 3 points to form a polygon
            triangles = triangulate_polygon(x, y)
        else:
            triangles = [[0, 1, 2]]

        # Bottom face
        # fig.add_trace(go.Mesh3d(
        #     x=x,
        #     y=y,
        #     z=[0] * len(x),
        #     i=triangles[:, 0],
        #     j=triangles[:, 1],
        #     k=triangles[:, 2],
        #     color='red',
        #     opacity=0.50
        # ))

        # Top face
        # fig.add_trace(go.Mesh3d(
        #     x=x,
        #     y=y,
        #     z=z,
        #     i=triangles[:, 0],
        #     j=triangles[:, 1],
        #     k=triangles[:, 2],
        #     color='red',
        #     opacity=0.50
        # ))

        # Side faces
        for i in range(len(x)):
            j = (i + 1) % len(x)
            fig.add_trace(go.Mesh3d(
                x=[x[i], x[j], x[j], x[i]],
                y=[y[i], y[j], y[j], y[i]],
                z=[0, 0, z[j], z[i]],
                i=[0, 0],
                j=[1, 2],
                k=[2, 3],
                color='red',
                opacity=0.50
            ))

        # Add borders for the top face
        fig.add_trace(go.Scatter3d(
            x=x + [x[0]],
            y=y + [y[0]],
            z=z + [z[0]],
            mode='lines',
            line=dict(color='black', width=3)
        ))

        # Add borders for the bottom face
        fig.add_trace(go.Scatter3d(
            x=x + [x[0]],
            y=y + [y[0]],
            z=[0] * (len(x) + 1),
            mode='lines',
            line=dict(color='black', width=3)
        ))

        # Add borders for the sides
        for i in range(len(x)):
            j = (i + 1) % len(x)
            fig.add_trace(go.Scatter3d(
                x=[x[i], x[i], x[j], x[j], x[i]],
                y=[y[i], y[i], y[j], y[j], y[i]],
                z=[0, z[i], z[j], 0, 0],
                mode='lines',
                line=dict(color='black', width=3)
            ))

    # Delivery points
    loc_x = [i for i, j, k in delivery_locations]
    loc_y = [j for i, j, k in delivery_locations]
    loc_z = [k for i, j, k in delivery_locations]

    fig.add_trace(go.Scatter3d(
        x=loc_x,
        y=loc_y,
        z=loc_z,
        mode='markers',
        marker=dict(size=5, color='blue')
    ))

    # Plot dp
    if dp:
        fig.add_trace(go.Scatter3d(
            x=[dp[0]],
            y=[dp[1]],
            z=[dp[2]],
            mode='markers',
            marker=dict(size=8, color='orange')
        ))

    # Plotting route
    if route:
        x_axis = [i for i, j, k in route]
        y_axis = [j for i, j, k in route]
        z_axis = [k for i, j, k in route]

        fig.add_trace(go.Scatter3d(
            x=x_axis,
            y=y_axis,
            z=z_axis,
            mode='lines',
            line=dict(color='gray', width=8)
        ))

    # Update layout for better view
    fig.update_layout(
        width = 800,
        height = 800,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Height',
            aspectmode='cube',
    ))

    # Show plot
    fig.show()

In [27]:
obstacles = {
    0 : [[27.81, 90.75,3],[28.74, 91.42, 3],[27.80, 93.15, 3], [26.81, 92.53,3], [27.81, 90.75,3]],
    1 : [[23.11, 91.25,3], [24.81, 92.40,3], [24.21, 93.45, 3], [22.51, 92.35, 3],[23.11, 91.25, 3]],
    2 : [[24.55, 88.60,10],[25.96, 89.55,10],[25.01, 91.35, 10],[23.61, 90.32, 10],[24.55, 88.60, 10]],
    3 : [[25.60, 88.45,10], [27.42, 88.45,10],[26.92, 89.35,10],[25.60, 88.45,10]],
    4 : [[27.61, 88.45,10], [28.81, 88.45,10],[29.12, 88.66,10],[28.02, 90.45,10],[26.90, 89.57,10],[27.61, 88.45,10]],
    5 : [[25.33, 92.62,10], [25.86, 92.95,10], [25.21, 94.15,10], [24.69, 93.78,10], [25.33, 92.63,10]],
    6 : [[26.11, 93.05,3], [26.62, 93.35,3], [25.81, 94.65,3], [25.39, 94.35,3], [26.11, 93.05,3]],
    7 : [[26.72, 93.65,10], [27.32, 94.05,10], [26.69, 95.18,10], [26.10, 94.83,10], [26.72, 93.65,10]],
    8 : [[25.81, 95.40,10],[26.29, 95.70,10],[26.01, 96.20,10],[26.25, 96.37,10],[25.93, 96.94,10],[25.80, 96.84,10], [25.69, 97.01,10],[25.11, 96.65,10],[25.81, 95.40,10]],
    9 : [[23.90, 94.15,3], [24.91, 94.81,3], [24.48, 95.57,3],[23.48, 94.95,3],[23.90, 94.15,3]],
    10 : [[23.71, 98.67,10], [23.91, 98.26,10], [24.41, 98.67,10],[23.71, 98.67,10]],
    11 : [[31.81, 90.35,10], [32.51, 90.85,10], [32.21, 91.55,10], [32.03, 91.85,10], [34.31, 93.45,10], [32.69, 96.27,10],[30.99, 95.15,10], [32.21, 92.93,10],[30.89, 91.97,10],[31.81, 90.35,10]],
    12 : [[30.21, 96.00,10],[31.85, 97.11,10], [31.02, 98.55,10], [30.50, 98.26,10], [30.31, 98.53,10], [30.51, 98.67,10],[27.81, 98.67,10], [27.21, 98.28,10], [27.73, 97.43,10],[28.95, 98.25,10],[30.21, 96.00,10]]
}

source = [27.55470577587296, 93.92084405438013, 0]
dest = [[25.56,92.34,0]]
route = [source,[source[0], source[1], 5],[dest[0][0], dest[0][1],5], dest[0]]

In [28]:
initial_plot(dest, obstacles, dp = source, route=route)

In [29]:

def create_plane(coords):
    fig = go.Figure()

    # Extract x, y, z coordinates from coords
    x = [point[0] for point in coords]
    y = [point[1] for point in coords]
    z = [point[2] for point in coords]

    # Create Mesh3d to fill the shape
    fig.add_trace(go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='lines',
            marker=dict(size=8, color='blue')
    ))

    fig.add_trace(go.Mesh3d(
        x=x,
        y=y,
        z=z,
        opacity=0.80,
    ))

    fig.show()

In [30]:
coords = [[31.81, 90.35,10], [32.51, 90.85,10], [32.21, 91.55,10], [32.03, 91.85,10], [34.31, 93.45,10], [32.69, 96.27,10],[30.99, 95.15,10], [32.21, 92.93,10],[30.89, 91.97,10],[31.81, 90.35,10]]

create_plane(coords)